# Chemical Intervention on Rat

Experiment Narrative:
---------------------

Male Fisher F344 rats purchased from Charles River were treated with 3 commonly used painkillers, namely acetylsalicilic acid, acetaminophen and ibuprofen, at 2 distinct dose levels, delivered per os.
Equal number of animals (n=5) were allocated to each of the group defined by a compound, dose level and duration post exposure combination.
Following sacrifice performed by cervical dislocation and exsanguination preceded by anesthesia (ketamine and xylazine solution), blood and kidney specimens were collected .
Total RNA were extracted from Kidney samples and mRNA sequencing (transcription profiling) was performed using paired-end libraries on Illumina sequencing platform using an Illumina HiSeq 2000 instrument.
Blood samples were collected at sacrifice time and immediately placed in precooled 60 percent methanol ammonium bicarbonate buffer to quench cellular metabolism. Blood Metabolites were separated in water-soluble and lipophilic fractions. 
Metabolite profiling was performed on the polar metabolite fraction only, using flow injection analysis (FIA) mass spectrometry on an Agilent 6550 iFunnel Q-TOF Mass Sprectrometry platform. Each fraction was injected twice and data were acquired in both ionization modes (positive mode and negative mode).
Raw data files were saved in native instrument format and later converted to HUPO-PSI standard format for mass spectrometry.


In [460]:
from ipywidgets import (RadioButtons, VBox, HBox, Layout, Label, Checkbox, Text, IntSlider)
from qgrid import show_grid
label_layout = Layout(width='100%')
from isatools.create.models import *
from isatools.model import Investigation
from isatools.isatab import dump_tables_to_dataframes as dumpdf
import qgrid
qgrid.nbinstall(overwrite=True)

## Study Design Description and Treatment Plans

### Study design type

Based on the textual description provided, specify the type of study being conducted, identify the different independent variables and their respective levels.

### Intervention study

If specifying an intervention study, please answer the following:
 - Are study subjects exposed to a single intervention or to multiple intervention?
  


In [461]:
study_design = InterventionStudyDesign()

In [462]:
design_type = '' 
print('Design type: {}'.format(design_type))

Design type: 


#### Study design - intervention types

In the context of the experiment, identify the independent variables and their associated levels.
How many treatment group are defined? 


In [463]:
level_uis = []
agent_levels = Text(
        description='Agent:',
        value='a,b,c',
        placeholder='',

    )
dose_levels = Text(
        description='Dose levels:',
        value='1,2',
        placeholder='',

    )
duration_of_exposure_levels = Text(
        description='Duration post exposure:',
        value='t1,t2,t3',
        placeholder='',
)

# Computing the Number of Unique Treatment Groups/Study Groups: 

Using the ISA-API create mode, determine the total number of treatment groups.

In [464]:
factory = TreatmentFactory(intervention_type=INTERVENTIONS['CHEMICAL'], factors=BASE_FACTORS)
for agent_level in agent_levels.value.split(','):
    factory.add_factor_value(BASE_FACTORS[0], agent_level.strip())
for dose_level in dose_levels.value.split(','):
    factory.add_factor_value(BASE_FACTORS[1], dose_level.strip())
for duration_of_exposure_level in duration_of_exposure_levels.value.split(','):
    factory.add_factor_value(BASE_FACTORS[2], duration_of_exposure_level.strip())
    
print('Number of study groups (treatment groups): {}'.format(len(factory.compute_full_factorial_design())))

treatment_sequence = TreatmentSequence(ranked_treatments=factory.compute_full_factorial_design())

Number of study groups (treatment groups): 18


# Setting Study Group Sizes:

Next, specify if all study groups have the same size (in other words, do they have the same number of subjects?)

What type of replication this is? 

From the following list of terms, select which that apply to the particular study:

In [465]:
group_balanced = RadioButtons(options=['Balanced', 'Unbalanced'], value='Balanced', disabled=False)
VBox([Label('Are study groups balanced?', layout=label_layout), group_balanced])

Provide the number of subject per study group:

In [466]:
group_size = IntSlider(value=3, min=0, max=100, step=1, description='Group size:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
group_size

In [467]:
group_size.value="3"
plan = SampleAssayPlan(group_size=group_size.value)

# Establishing the sample collection plan

For each study subject, determine the number of unique specimens collected over the course of the study. How many times each of the samples have been collected?

In [471]:
collection_plan = {"kidney":1, "blood":1} #dictionary key should be a 'biological specimen' (e.g. brain), dictionary value should be an integer

for key in collection_plan:
    sample_type = key
    collected_sample_type_nb = collection_plan[key]
    #print(key, collected_sample_type_nb)
    plan.add_sample_type(sample_type)
    plan.add_sample_plan_record(sample_type,collected_sample_type_nb)
    isa_object_factory = IsaModelObjectFactory(plan, treatment_sequence)

## Create an ISA Study Table reflecting the study design and the sampling plan:

In the following section, the task is to build ISA objects from the sample plan and render the study-sample table.


*Check state of the Sample Assay Plan after entering sample planning information:*

In [472]:
isa_investigation = Investigation(identifier='inv_dtp-exercise-1')
isa_study = isa_object_factory.create_study_from_plan()
isa_study.filename = 's_study.txt'
isa_investigation.studies = [isa_study]
dataframes = dumpdf(isa_investigation)
sample_table = next(iter(dataframes.values()))
show_grid(sample_table)

In [473]:
print('Total rows generated: {}'.format(len(sample_table)))

Total rows generated: 108


# Saving key study design information as a JSON document.

This section is meant to show how to serialize key study design descriptors in a compact document serialized in format format. 
Why is this relevant? How would you use this feature. List 3 possible uses.

In [474]:
import json
from isatools.create.models import SampleAssayPlanEncoder
print(json.dumps(plan, cls=SampleAssayPlanEncoder, sort_keys=True, indent=4, separators=(',', ': ')))

{
    "assay_plan": [],
    "assay_types": [],
    "group_size": 3,
    "sample_plan": [
        {
            "sample_type": "kidney",
            "sampling_size": 1
        },
        {
            "sample_type": "blood",
            "sampling_size": 1
        }
    ],
    "sample_qc_plan": [],
    "sample_types": [
        "blood",
        "kidney"
    ]
}


## Documenting the Assay and Data Acquisition Plans:

From the textual description, identify the Response variables / Dependent Variables

### Binding assay technology types to sample type: 
Check against ISA configuration to obtain vetted values for Measurement Type and Technology type.
An ISA configuration can be accessed from: https://github.com/ISA-tools/Configuration-Files/tree/master/isaconfig-default_v2015-07-02

In [475]:
assay_types = set()

assay_type1 = AssayType(measurement_type='transcription profiling', technology_type='nucleotide sequencing')
assay_type2 = AssayType(measurement_type='metabolite profiling', technology_type='mass spectrometry')

assay_types.add(assay_type1)
assay_types.add(assay_type2)
for x in assay_types:
        print(x.measurement_type.term," using ", x.technology_type.term)


metabolite profiling  using  mass spectrometry
transcription profiling  using  nucleotide sequencing


## Assay Specific Descriptors
Each data acquisition modality comes with its own set of parameters which may be used to capture the underlying workflow graph modifications. This section of the exercise aims to show how to deal with workflows specific to sequencing applications and mass spectrometry ones.

### Generate ISA model objects from the assay plan and render the assay table

In [476]:
# Starting with DNA sequencing assay workflow: 
# Setting the type of library and the number of runs per sample.

# This bit is meant to introduce you to iwidgets, a library for ipython allowing user interaction (not implemented here)
ngs_technical_replicates = IntSlider(value=2, min=0, max=5, step=1, description='Technical repeats:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
print(ngs_technical_replicates.value)
ngs_technical_replicates



2


### Check state of Sample Assay Plan after entering assay plan information:

### Dealing with Next Generation Sequencing Data Acquisition Plan

In [478]:
sequencing_instruments = set()
sequencing_instruments.add('Illumina HiSeq 2000') # check the experiment description to obtain value.

top_mods_seq = DNASeqAssayTopologyModifiers(technical_replicates=ngs_technical_replicates.value, instruments=sequencing_instruments,distinct_libraries=2)

print('Technical replicates: {}'.format(top_mods_seq.technical_replicates))

assay_type1.topology_modifiers = top_mods_seq

plan.add_assay_type(assay_type1)
# sample_type = plan.sample_types[]
for element in plan.sample_types:
    if element.value.term == 'kidney':
        print(element.value.term)
        plan.add_assay_plan_record(element.value.term, assay_type1)
        assay_plan = next(iter(plan.assay_plan))
    elif element.value.term == 'blood':
        print("not for this assay type")
    else:
        print(element.value.term)

# if sample_type not in plan.sample_types:
#     print(sample_type + " Not in sample types")
#else:    
#    plan.add_assay_plan_record(sample_type, assay_type1)  # check the experiment description to obtain value.
   

print('Added assay plan: {0} -> {1}/{2}'.format(assay_plan[0].value.term, assay_plan[1].measurement_type.term, assay_plan[1].technology_type.term))
if len(top_mods_seq.instruments) > 0:
    print('Instruments: {}'.format(list(top_mods_seq.instruments)))

Technical replicates: 2
kidney
not for this assay type
Added assay plan: kidney -> transcription profiling/nucleotide sequencing
Instruments: ['Illumina HiSeq 2000']


In [479]:
print(json.dumps(plan, cls=SampleAssayPlanEncoder, sort_keys=True, indent=4, separators=(',', ': ')))

{
    "assay_plan": [
        {
            "assay_type": {
                "measurement_type": "transcription profiling",
                "technology_type": "nucleotide sequencing",
                "topology_modifiers": {
                    "distinct_libraries": 2,
                    "instruments": [
                        "Illumina HiSeq 2000"
                    ],
                    "technical_replicates": 2
                }
            },
            "sample_type": "kidney"
        }
    ],
    "assay_types": [
        {
            "measurement_type": "transcription profiling",
            "technology_type": "nucleotide sequencing",
            "topology_modifiers": {
                "distinct_libraries": 2,
                "instruments": [
                    "Illumina HiSeq 2000"
                ],
                "technical_replicates": 2
            }
        }
    ],
    "group_size": 3,
    "sample_plan": [
        {
            "sample_type": "kidney",
            "sa

### Dealing with Mass Spectrometry Data Acquisition Plan

In [480]:
chromatography_instruments = set()
ms_instruments = set()
injection_modes = set()
acquisition_modes = set()

ms_instruments.add('agilent') # check the experiment description to obtain value.
injection_modes.add('FIA')  # check the experiment description to obtain value.
acquisition_modes.add('positive') # check the experiment description to obtain value.
acquisition_modes.add('negative') # check the experiment description to obtain value.
ms_tech_rep = 1 # check the experiment description to obtain value.
top_mods_ms = MSAssayTopologyModifiers(technical_replicates=ms_tech_rep, injection_modes=injection_modes, acquisition_modes=acquisition_modes, instruments=ms_instruments, chromatography_instruments=chromatography_instruments)
assay_type2.topology_modifiers = top_mods_ms

if len(top_mods_ms.chromatography_instruments) > 0:
    print('Chromatography instruments: {}'.format(list(top_mods_ms.chromatography_instruments)))
else:
    print('no chromatography used or no information supplied')

if len(top_mods_ms.instruments) > 0:
    print('Data acquisition instruments: {}'.format(list(top_mods_ms.instruments)))    
if len(top_mods_ms.injection_modes) > 0:
    print('Injection modes: {}'.format(list(top_mods_ms.injection_modes)))
if len(top_mods_ms.acquisition_modes) > 0:
    print('Acquisition modes: {}'.format(list(top_mods_ms.acquisition_modes)))


plan.add_assay_type(assay_type2)
plan.add_assay_plan_record("blood", assay_type2)

assay_plan = next(iter(plan.assay_plan))



no chromatography used or no information supplied
Data acquisition instruments: ['agilent']
Injection modes: ['FIA']
Acquisition modes: ['positive', 'negative']


In [481]:
print(json.dumps(plan, cls=SampleAssayPlanEncoder, sort_keys=True, indent=4, separators=(',', ': ')))

{
    "assay_plan": [
        {
            "assay_type": {
                "measurement_type": "transcription profiling",
                "technology_type": "nucleotide sequencing",
                "topology_modifiers": {
                    "distinct_libraries": 2,
                    "instruments": [
                        "Illumina HiSeq 2000"
                    ],
                    "technical_replicates": 2
                }
            },
            "sample_type": "kidney"
        },
        {
            "assay_type": {
                "measurement_type": "metabolite profiling",
                "technology_type": "mass spectrometry",
                "topology_modifiers": {
                    "acquisition_modes": [
                        "negative",
                        "positive"
                    ],
                    "chromatography_instruments": [],
                    "injection_modes": [
                        "FIA"
                    ],
                    "

In [482]:
isa_investigation.studies = [isa_object_factory.create_assays_from_plan()]
for assay in isa_investigation.studies[-1].assays:
    print('Assay generated: {0}, {1} samples, {2} processes, {3} data files'
          .format(assay.filename, len(assay.samples), len(assay.process_sequence), len(assay.data_files)))
dataframes = dumpdf(isa_investigation)

A protocol with name "metabolite extraction" has already been declared in the study
Assay generated: a_kidney_dnaseq_Illumina HiSeq 2000_assay.txt, 54 samples, 324 processes, 108 data files
Assay generated: a_blood_ms_FIA_positive_assay.txt, 54 samples, 108 processes, 54 data files
Assay generated: a_blood_ms_FIA_negative_assay.txt, 54 samples, 108 processes, 54 data files


In [483]:
show_grid(dataframes[next(iter(dataframes.keys()))])

In [484]:
show_grid(dataframes['a_blood_ms_FIA_positive_assay.txt'])

In [485]:
show_grid(dataframes['a_blood_ms_FIA_negative_assay.txt'])